## Catboost + Hyperplot
Итоговый скор: `metric = NDCG:top=10;type=Exp score = 0.512`
на параметрах
```
{'bagging_temperature': 0.7825876839248045,
 'border_count': 128,
 'depth': 8,
 'eval_metric': 'NDCG:top=10;type=Exp',
 'grow_policy': 'Depthwise',
 'iterations': 5870,
 'l2_leaf_reg': 5.930424236293685,
 'learning_rate': 0.09987631032212624,
 'loss_function': 'YetiRank',
 'random_seed': 42,
 'task_type': 'GPU',
 'use_best_model': True,
 'verbose': 200}
```
Обучение и исследования проводились на платформе kaggle

In [1]:
import pathlib
from timeit import default_timer as timer

from functools import partial

import numpy as np
import pandas as pd

import catboost
from catboost import datasets, utils, CatBoostRanker
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from hyperopt.pyll import scope

## Сборка данных

In [2]:
!ls /kaggle/input/mslr-web10k/Fold1

test.txt  train.txt  vali.txt


In [3]:
cnt = 10 # to use in filename for optim model and params

In [4]:
def generate_column_names(num_features):
    """Generates column names for LETOR-like datasets"""
    columns = ['label', 'qid']
    for i in range(num_features):
        column = f"feature_{i+1}"
        columns.append(column)
    return columns
    
def load_svmlight_file(input_file, max_num_lines=0):
    """Loads dataset split in SVM-Light format"""
    def _parse_field(field):
        parts = field.split(':')
        if len(parts) != 2:
            raise Exception(f"invalid number of parts in field {field}")
        return parts

    num_features = 136
    exp_num_fields = num_features + 2
    num_lines = 0
    X = []
    with open(input_file, 'rt') as f:
        for line in f:
            try:
                num_lines += 1
                                  
                # Parse into fields
                fields = line.rstrip().split(' ')
                num_fields = len(fields)
                if num_fields != exp_num_fields:
                    raise Exception(f"invalid number of fields {num_fields}")
    
                # Parse every field
                x = np.zeros(exp_num_fields, dtype=np.float32)
                label = int(fields[0])
                x[0] = label
                _, qid_str = _parse_field(fields[1])
                qid = int(qid_str)
                x[1] = qid
                for i, field in enumerate(fields[2:]):
                    _, feature_str = _parse_field(field)
                    x[i+2] = float(feature_str)
    
                # Add new object
                X.append(x)
                if num_lines % 50000 == 0:
                    print(f"Loaded {num_lines} lines...")
                if max_num_lines > 0 and num_lines == max_num_lines:
                    print(f"WARNING: stop loading, line limit reached: max_num_lines = {max_num_lines} input_file = {input_file}")
                    break
            except Exception as e:
                raise Exception(f"error at line {num_lines} in {input_file}") from e
    
    # To pandas
    df = pd.DataFrame(X, columns=generate_column_names(num_features))
    print(f"Loaded SVM-Light file {input_file}")
    return df

In [5]:
fold_dir = pathlib.Path("/kaggle/input/mslr-web10k/Fold1")

df_train = load_svmlight_file(fold_dir.joinpath("train.txt"))
df_valid = load_svmlight_file(fold_dir.joinpath("vali.txt"))
df_test = load_svmlight_file(fold_dir.joinpath("test.txt"))
print(f"Dataset loaded from fold_dir {fold_dir}")

Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded 250000 lines...
Loaded 300000 lines...
Loaded 350000 lines...
Loaded 400000 lines...
Loaded 450000 lines...
Loaded 500000 lines...
Loaded 550000 lines...
Loaded 600000 lines...
Loaded 650000 lines...
Loaded 700000 lines...
Loaded SVM-Light file /kaggle/input/mslr-web10k/Fold1/train.txt
Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded SVM-Light file /kaggle/input/mslr-web10k/Fold1/vali.txt
Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded SVM-Light file /kaggle/input/mslr-web10k/Fold1/test.txt
Dataset loaded from fold_dir /kaggle/input/mslr-web10k/Fold1


In [6]:
print(df_train.head(5))

   label  qid  feature_1  feature_2  feature_3  feature_4  feature_5  \
0    2.0  1.0        3.0        3.0        0.0        0.0        3.0   
1    2.0  1.0        3.0        0.0        3.0        0.0        3.0   
2    0.0  1.0        3.0        0.0        2.0        0.0        3.0   
3    2.0  1.0        3.0        0.0        3.0        0.0        3.0   
4    1.0  1.0        3.0        0.0        3.0        0.0        3.0   

   feature_6  feature_7  feature_8  ...  feature_127  feature_128  \
0        1.0        1.0   0.000000  ...         62.0   11089534.0   
1        1.0        0.0   1.000000  ...         54.0   11089534.0   
2        1.0        0.0   0.666667  ...         45.0          3.0   
3        1.0        0.0   1.000000  ...         56.0   11089534.0   
4        1.0        0.0   1.000000  ...         64.0          5.0   

   feature_129  feature_130  feature_131  feature_132  feature_133  \
0          2.0        116.0      64034.0         13.0          3.0   
1          2

In [7]:
print(df_valid.head(5))

   label   qid  feature_1  feature_2  feature_3  feature_4  feature_5  \
0    0.0  10.0        2.0        0.0        0.0        0.0        2.0   
1    0.0  10.0        1.0        0.0        1.0        3.0        3.0   
2    1.0  10.0        3.0        0.0        3.0        0.0        3.0   
3    0.0  10.0        3.0        0.0        2.0        0.0        3.0   
4    1.0  10.0        3.0        0.0        3.0        0.0        3.0   

   feature_6  feature_7  feature_8  ...  feature_127  feature_128  \
0   0.666667        0.0   0.000000  ...         45.0          1.0   
1   0.333333        0.0   0.333333  ...         76.0          0.0   
2   1.000000        0.0   1.000000  ...         73.0          0.0   
3   1.000000        0.0   0.666667  ...         54.0          8.0   
4   1.000000        0.0   1.000000  ...         36.0          6.0   

   feature_129  feature_130  feature_131  feature_132  feature_133  \
0          0.0        117.0      55115.0          7.0          2.0   
1     

In [8]:
print(df_test.head(5))

   label   qid  feature_1  feature_2  feature_3  feature_4  feature_5  \
0    2.0  13.0        2.0        0.0        2.0        1.0        2.0   
1    1.0  13.0        2.0        0.0        0.0        0.0        2.0   
2    3.0  13.0        2.0        0.0        1.0        0.0        2.0   
3    1.0  13.0        2.0        0.0        2.0        1.0        2.0   
4    0.0  13.0        1.0        0.0        0.0        0.0        1.0   

   feature_6  feature_7  feature_8  ...  feature_127  feature_128  \
0        1.0        0.0        1.0  ...         35.0          1.0   
1        1.0        0.0        0.0  ...         17.0         93.0   
2        1.0        0.0        0.5  ...         19.0          0.0   
3        1.0        0.0        1.0  ...         50.0      81775.0   
4        0.5        0.0        0.0  ...         24.0          0.0   

   feature_129  feature_130  feature_131  feature_132  feature_133  \
0          0.0        266.0      25070.0         28.0          7.0   
1     

In [9]:
df_train.sort_values(by='qid', inplace=True)
df_valid.sort_values(by='qid', inplace=True)
df_test.sort_values(by='qid', inplace=True)

In [10]:
df_train['qid'].is_monotonic_increasing, df_valid['qid'].is_monotonic_increasing, df_test['qid'].is_monotonic_increasing

(True, True, True)

In [11]:
qid_train = set(df_train['qid'].unique().astype('int'))
qid_valid = set(df_valid['qid'].unique().astype('int'))
qid_test = set(df_test['qid'].unique().astype('int'))
print(set.intersection(qid_train, qid_valid))
print(set.intersection(qid_train, qid_test))
print(set.intersection(qid_test, qid_valid))

set()
set()
set()


In [12]:
class Model:
    def __init__(self, optim_params=None):
        # self.verbose = verbose
        self.optim_params = optim_params
        self.model = None
        if self.optim_params is not None:
            self.model = catboost.CatBoostRanker(**optim_params)
            
    def _create_pool(self, df, return_yqx=False):
        y = df["label"].to_numpy()
        q = df["qid"].astype('int32').to_numpy()
        X = df.drop(["label", "qid"], axis=1).to_numpy()
        pool = catboost.Pool(data=X, label=y, group_id=q)
        # print(pool.get_group_id_hash())
        if return_yqx:
            return pool, (y, q, X)
        return pool

    def fit(self, df_train, df_valid):
        pool_train = self._create_pool(df_train)
        pool_valid = self._create_pool(df_valid)

        # there is no optim params
        if self.model is None:
            self.model = CatBoostRanker()

        self.model.fit(pool_train, eval_set=pool_valid, verbose=200, use_best_model=True)

    def predict(self, df_test):
        pool_test = self._create_pool(df_test)
        return self.model.predict(pool_test)

Функция создания тренировочного Pool для CatBoost:

In [13]:
 def create_pool(df, return_yqx=False):
        y = df["label"].to_numpy()
        q = df["qid"].to_numpy().astype('uint32')
        X = df.drop(["label", "qid"], axis=1).to_numpy()
        pool = catboost.Pool(data=X, label=y, group_id=q)
        # print(pool.get_group_id_hash())
        if return_yqx:
            return pool, (y, q, X)
        return pool

### Будем подбирать параметры с помощью hyperopt.
Классический шаблон:

In [14]:
def run_hyperopt(search_space):
    def objective(params, df_train, df_valid):
        model = CatBoostRanker(**params)
        pool_valid, (y_true, group_id, _) = create_pool(df_valid, return_yqx=True)
        pool_train = create_pool(df_train)

        model.fit(pool_train, eval_set=pool_valid, use_best_model=True)
        y_hat = model.predict(pool_valid)
        score = utils.eval_metric(y_true, y_hat, 'NDCG:top=10;type=Exp', group_id=group_id)[0]

        return {'loss': 1-score, 'params': params, 'status': STATUS_OK}

    trials = Trials()

    best_params = fmin(fn=partial(objective, df_train=df_train, df_valid=df_valid),
                     space=search_space,
                     algo=tpe.suggest,
                     max_evals=75,
                     trials=trials,
                     rstate=np.random.default_rng(42))

    return trials

In [15]:
search_space_1 = {
            'learning_rate': hp.loguniform(label='learning_rate', low=-4*np.log(10), high=-1 * np.log(10)), # lr = [0.001, 0.001, ..., 1]
            'bagging_temperature': hp.uniform('bagging_temperature', 0.7, 1.1),
            'depth': hp.randint('depth', 4, 10),
            'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 20),
            'use_best_model': True,
            'iterations' : 1200,
            'verbose' : 200,
            'random_seed' : 42,
            'border_count' : 128,
            'grow_policy': 'Depthwise',
            'loss_function': 'YetiRank',
            'eval_metric': 'NDCG:top=10;type=Exp',
            'task_type': 'GPU',
            }
trial = run_hyperopt(search_space_1)

  0%|          | 0/75 [00:00<?, ?trial/s, best loss=?]

Default metric period is 5 because NDCG
 is/are not implemented for GPU

Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time

Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time



0:	test: 0.3380666	best: 0.3380666 (0)	total: 280ms	remaining: 5m 35s

200:	test: 0.4686512	best: 0.4686512 (200)	total: 5.73s	remaining: 28.5s

400:	test: 0.4814675	best: 0.4814675 (400)	total: 11.8s	remaining: 23.6s

600:	test: 0.4899037	best: 0.4899037 (600)	total: 17.8s	remaining: 17.7s

800:	test: 0.4963812	best: 0.4964533 (790)	total: 23s	remaining: 11.5s

1000:	test: 0.4991445	best: 0.4991445 (1000)	total: 28.2s	remaining: 5.6s

1199:	test: 0.5026114	best: 0.5026130 (1180)	total: 33.2s	remaining: 0us

bestTest = 0.5026129584                               

bestIteration = 1180                                  

Shrink model to first 1181 iterations.                
  1%|▏         | 1/75 [00:39<49:03, 39.77s/trial, best loss: 0.49738518187822023]

Default metric period is 5 because NDCG
 is/are not implemented for GPU

Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time

Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time



0:	test: 0.3637648	best: 0.3637648 (0)	total: 120ms	remaining: 2m 24s            

200:	test: 0.4583438	best: 0.4584489 (175)	total: 10.3s	remaining: 51.4s         

400:	test: 0.4593791	best: 0.4596560 (395)	total: 20.4s	remaining: 40.7s         

600:	test: 0.4617921	best: 0.4617921 (600)	total: 30.2s	remaining: 30.1s         

800:	test: 0.4621026	best: 0.4621026 (800)	total: 41.1s	remaining: 20.5s         

1000:	test: 0.4632438	best: 0.4634762 (985)	total: 51.1s	remaining: 10.2s        

1199:	test: 0.4641693	best: 0.4642185 (1140)	total: 1m	remaining: 0us            

bestTest = 0.4642185448                                                          

bestIteration = 1140                                                             

Shrink model to first 1141 iterations.                                           
  3%|▎         | 2/75 [01:49<1:10:07, 57.64s/trial, best loss: 0.49738518187822023]

...

0:	test: 0.3678055	best: 0.3678055 (0)	total: 95.4ms	remaining: 1m 54s            

200:	test: 0.4860074	best: 0.4860074 (200)	total: 8.78s	remaining: 43.6s          

400:	test: 0.4968319	best: 0.4968319 (400)	total: 17.2s	remaining: 34.3s          

600:	test: 0.5037653	best: 0.5037653 (600)	total: 24.6s	remaining: 24.5s          

800:	test: 0.5082069	best: 0.5082069 (800)	total: 32s	remaining: 15.9s            

1000:	test: 0.5099754	best: 0.5099754 (1000)	total: 39.9s	remaining: 7.93s        

1199:	test: 0.5108163	best: 0.5108163 (1199)	total: 47.7s	remaining: 0us          

bestTest = 0.510816253                                                            

bestIteration = 1199                                                              

 97%|█████████▋| 73/75 [59:48<01:50, 55.01s/trial, best loss: 0.48406651150023994]

Default metric period is 5 because NDCG
 is/are not implemented for GPU

Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time

Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time



0:	test: 0.3077999	best: 0.3077999 (0)	total: 60.6ms	remaining: 1m 12s            

200:	test: 0.4779549	best: 0.4779549 (200)	total: 4.91s	remaining: 24.4s          

400:	test: 0.4909911	best: 0.4909911 (400)	total: 9.42s	remaining: 18.8s          

600:	test: 0.4976519	best: 0.4976519 (600)	total: 13.5s	remaining: 13.4s            

800:	test: 0.4994051	best: 0.4995122 (790)	total: 17.9s	remaining: 8.89s            

1000:	test: 0.5011181	best: 0.5011281 (995)	total: 22.1s	remaining: 4.4s            

1199:	test: 0.5023841	best: 0.5024190 (1185)	total: 26.8s	remaining: 0us            

bestTest = 0.5024190065                                                             

bestIteration = 1185                                                                

Shrink model to first 1186 iterations.                                              
 99%|█████████▊| 74/75 [1:00:21<00:48, 48.15s/trial, best loss: 0.48406651150023994]

Default metric period is 5 because NDCG
 is/are not implemented for GPU

Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time

Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time



0:	test: 0.3692817	best: 0.3692817 (0)	total: 97ms	remaining: 1m 56s                

200:	test: 0.5010744	best: 0.5010744 (200)	total: 8.38s	remaining: 41.7s            

400:	test: 0.5088049	best: 0.5088969 (390)	total: 16.6s	remaining: 33.1s            

600:	test: 0.5130001	best: 0.5130763 (595)	total: 24.7s	remaining: 24.6s            

800:	test: 0.5141427	best: 0.5144570 (795)	total: 32.8s	remaining: 16.3s            

1000:	test: 0.5160005	best: 0.5160415 (990)	total: 40.5s	remaining: 8.05s           

1199:	test: 0.5169105	best: 0.5173019 (1160)	total: 48.2s	remaining: 0us            

bestTest = 0.5173019327                                                             

bestIteration = 1160                                                                

Shrink model to first 1161 iterations.                                              
100%|██████████| 75/75 [1:01:17<00:00, 49.03s/trial, best loss: 0.48269806729754394]


In [16]:
print("NDCG:top=10:type=exp score: ", 1 - trial.best_trial['result']['loss'])
trial.best_trial['result']['params']

NDCG:top=10:type=exp score:  0.5173019327024561


{'bagging_temperature': 0.7825876839248045,
 'border_count': 128,
 'depth': 8,
 'eval_metric': 'NDCG:top=10;type=Exp',
 'grow_policy': 'Depthwise',
 'iterations': 1200,
 'l2_leaf_reg': 5.930424236293685,
 'learning_rate': 0.09987631032212624,
 'loss_function': 'YetiRank',
 'random_seed': 42,
 'task_type': 'GPU',
 'use_best_model': True,
 'verbose': 200}

In [17]:
optim_params = trial.best_trial['result']['params']

### Посмотрим, на какой итерации CatBoost получим лучший скор:

In [18]:
optim_params['iterations'] = 10000
optim_params['task_type'] = 'GPU'
optim_params['eval_metric'] = 'NDCG:top=10;type=Exp;hints=skip_train~false'

print(optim_params)
model_w_best_iter = Model(optim_params)
start = timer()
model_w_best_iter.fit(df_train, df_valid)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f}")

{'bagging_temperature': 0.7825876839248045, 'border_count': 128, 'depth': 8, 'eval_metric': 'NDCG:top=10;type=Exp;hints=skip_train~false', 'grow_policy': 'Depthwise', 'iterations': 10000, 'l2_leaf_reg': 5.930424236293685, 'learning_rate': 0.09987631032212624, 'loss_function': 'YetiRank', 'random_seed': 42, 'task_type': 'GPU', 'use_best_model': True, 'verbose': 200}


Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.3712595	test: 0.3692817	best: 0.3692817 (0)	total: 124ms	remaining: 20m 44s
200:	learn: 0.5197893	test: 0.4993557	best: 0.4993557 (200)	total: 8.27s	remaining: 6m 43s
400:	learn: 0.5371733	test: 0.5087834	best: 0.5087834 (400)	total: 16.3s	remaining: 6m 29s
600:	learn: 0.5479364	test: 0.5128494	best: 0.5128494 (600)	total: 24.1s	remaining: 6m 16s
800:	learn: 0.5558115	test: 0.5144299	best: 0.5145386 (780)	total: 32.2s	remaining: 6m 9s
1000:	learn: 0.5626171	test: 0.5151331	best: 0.5153869 (995)	total: 40.8s	remaining: 6m 6s
1200:	learn: 0.5690344	test: 0.5161120	best: 0.5161120 (1200)	total: 49.8s	remaining: 6m 5s
1400:	learn: 0.5745252	test: 0.5163086	best: 0.5165205 (1280)	total: 58.8s	remaining: 6m
1600:	learn: 0.5798511	test: 0.5161654	best: 0.5165205 (1280)	total: 1m 7s	remaining: 5m 56s
1800:	learn: 0.5855515	test: 0.5169852	best: 0.5169852 (1800)	total: 1m 16s	remaining: 5m 47s
2000:	learn: 0.5903342	test: 0.5171373	best: 0.5176673 (1895)	total: 1m 25s	remaining: 5m 

In [19]:
optim_params['iterations'] = model_w_best_iter.model.get_best_iteration() + 10
print("best score =", model_w_best_iter.model.best_score_)
print("on iteration it =", optim_params['iterations'])

best score = {'learn': {'NDCG:top=10;type=Exp': 0.71611548890575}, 'validation': {'NDCG:top=10;type=Exp': 0.519634936511361, 'NDCG:type=Base': 0.7989665932739829}}
on iteration it = 5870


## Обучение на оптимальных параметрах

Обучение можно проводить и на CPU, но оно

In [20]:
print(optim_params)
final_model = Model(optim_params)
start = timer()
final_model.fit(df_train, df_valid)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f}")

{'bagging_temperature': 0.7825876839248045, 'border_count': 128, 'depth': 8, 'eval_metric': 'NDCG:top=10;type=Exp;hints=skip_train~false', 'grow_policy': 'Depthwise', 'iterations': 5870, 'l2_leaf_reg': 5.930424236293685, 'learning_rate': 0.09987631032212624, 'loss_function': 'YetiRank', 'random_seed': 42, 'task_type': 'GPU', 'use_best_model': True, 'verbose': 200}


Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.3712595	test: 0.3692817	best: 0.3692817 (0)	total: 132ms	remaining: 12m 54s
200:	learn: 0.5196598	test: 0.5009675	best: 0.5009675 (200)	total: 9.16s	remaining: 4m 18s
400:	learn: 0.5368234	test: 0.5088413	best: 0.5088413 (400)	total: 18.1s	remaining: 4m 7s
600:	learn: 0.5479555	test: 0.5124156	best: 0.5124156 (600)	total: 26.5s	remaining: 3m 52s
800:	learn: 0.5561697	test: 0.5141585	best: 0.5141585 (800)	total: 35.4s	remaining: 3m 44s
1000:	learn: 0.5631231	test: 0.5148263	best: 0.5151669 (905)	total: 44.5s	remaining: 3m 36s
1200:	learn: 0.5689009	test: 0.5163800	best: 0.5165353 (1130)	total: 52.8s	remaining: 3m 25s
1400:	learn: 0.5749433	test: 0.5168487	best: 0.5171466 (1360)	total: 1m 1s	remaining: 3m 16s
1600:	learn: 0.5801164	test: 0.5172104	best: 0.5176451 (1480)	total: 1m 9s	remaining: 3m 5s
1800:	learn: 0.5854225	test: 0.5171367	best: 0.5178393 (1690)	total: 1m 17s	remaining: 2m 54s
2000:	learn: 0.5907345	test: 0.5174016	best: 0.5178393 (1690)	total: 1m 24s	remaining

In [21]:
def compute_metrics(y_true, y_hat, q):
    # List of metrics to evaluate
    eval_metrics = ['NDCG:top=10;type=Exp']
    
    for eval_metric in eval_metrics:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
    
        # Print scores
        print(f"metric = {eval_metric} score = {scores[0]:.3f}")
    
    return scores[0]

In [22]:
y_hat_final = final_model.predict(df_test)

y_test = df_test['label'].to_numpy()
q_test = df_test['qid'].to_numpy().astype('uint32')
    
# Compute metrics on test
metric = compute_metrics(y_test, y_hat_final, q_test)
print(metric)

metric = NDCG:top=10;type=Exp score = 0.512
0.5115967820307694


### Обучение на CPU

Для обучения на CPU достаточно выполнить блоки кода ниже (Основной результат был получен выше при обучении на GPU, тк это намного сокращает время оубчения и дает простор для тюнинга)

In [ ]:
optim_params['task_type'] = 'CPU'
print(optim_params)
cpu_model = Model(optim_params)
start = timer()
cpu_model.fit(df_train, df_valid)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f}")

In [ ]:
y_hat_final = cpu_model.predict(df_test)

y_test = df_test['label'].to_numpy()
q_test = df_test['qid'].to_numpy().astype('uint32')
    
# Compute metrics on test
metric = compute_metrics(y_test, y_hat_final, q_test)
print(metric)

### Сохранение финальных параметров

In [23]:
import pickle
cnt += 1
metric_str = f"{metric:.4f}"

filename = f"best_params_ctb_{cnt}_{metric_str}.pkl"
with open (filename, "wb") as f:
    pickle.dump(optim_params, f)